In [9]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn 

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import svm

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel

In [2]:
trainData = pd.read_csv("npf_train.csv")
class2 = np.array(["event"]*trainData.shape[0],dtype="object")
class2[trainData["class4"]=="nonevent"] = "nonevent"
trainData["class2"] = class2
trainData["class2"] = trainData["class2"].astype("category")

trainData = trainData.set_index("date")
trainData["class4"] = trainData["class4"].astype("category")
trainData = trainData.drop(["id", "partlybad"],axis=1)

trainData.insert(0, 'class22', trainData['class2'])
trainData = trainData.drop(columns='class2')
trainData = trainData.rename(columns={'class22': 'class2'})

trainData['Result'] = trainData['class2'].map(lambda x: x == "event")
trainData['Result'] = trainData['Result'].astype(int)
trainData.insert(0, 'resultV', trainData['Result'])
trainData = trainData.drop(columns='Result')
trainData = trainData.rename(columns={'resultV': 'result'})
trainData.head(3)

,result,class2,class4,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,CO2504.mean,...,T672.mean,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std
date,,,,,,,,,,,,,,,,,,,,,
2000-01-17,1,event,Ib,368.771711,0.310309,368.665658,0.305127,369.371184,0.333606,368.616053,...,-1.476016,0.250952,-0.899393,0.271648,2.492491,1.310880,0.031587,0.018122,0.000243,0.000035
2000-02-28,0,nonevent,nonevent,378.197295,1.001493,378.083089,1.025472,378.671311,1.017208,378.029508,...,1.479493,0.313460,2.033191,0.271090,0.295937,0.177836,0.005140,0.003552,0.003658,0.000940
2000-03-24,1,event,Ib,373.043158,0.749865,372.930066,0.728411,373.569735,0.835240,372.869868,...,-0.270844,2.037111,0.780422,2.334741,14.434789,8.627312,0.353743,0.272472,0.000591,0.000191


In [6]:
X = trainData.iloc[:, 3:-1]
y2 = trainData.iloc[:, 0]
y4 = trainData.iloc[:, 2]
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=0)
X4_train, X4_test, y4_train, y4_test = train_test_split(X, y4, test_size=0.2, random_state=0)

In [11]:
kernel_types = ["linear", "poly", "rbf", "sigmoid"]
resultsBinary = pd.DataFrame(columns = ['Kernel_type', 'Accuracy', 'Precision', 'Recall'])

for k in kernel_types:
    clf = svm.SVC(kernel=k) 
    clf.fit(X2_train, y2_train)
    y2_pred = clf.predict(X2_test)
    resultsBinary.loc[len(resultsBinary.index)] = [k, 
                                       metrics.accuracy_score(y2_test, y2_pred), 
                                       metrics.precision_score(y2_test, y2_pred),
                                       metrics.recall_score(y2_test, y2_pred)]
resultsBinary

,Kernel_type,Accuracy,Precision,Recall
0,linear,0.903226,0.900000,0.918367
1,poly,0.752688,0.740741,0.816327
2,rbf,0.752688,0.732143,0.836735
3,sigmoid,0.741935,0.727273,0.816327


In [14]:
kernel_types = ["linear", "poly", "rbf", "sigmoid"]
resultsMultiClass = pd.DataFrame(columns = ['Kernel_type', 'Accuracy'])

for k in kernel_types:
    clf = svm.SVC(kernel=k) 
    clf.fit(X4_train, y4_train)
    y4_pred = clf.predict(X4_test)
    resultsMultiClass.loc[len(resultsMultiClass.index)] = [k, metrics.accuracy_score(y4_test, y4_pred)]
    
resultsMultiClass

,Kernel_type,Accuracy
0,linear,0.677419
1,poly,0.526882
2,rbf,0.505376
3,sigmoid,0.473118


In [16]:
estimator = svm.SVC(kernel="linear")
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y2)
df2_bestFeatures = X.loc[: , selector.support_]
df2_bestFeatures

,H2O168.std,H2O42.mean,H2O42.std,H2O504.mean,H2O672.mean,H2O84.mean,H2O84.std,NOx168.std,NOx504.mean,NOx672.std,Pamb0.std,RHIRGA336.mean,RHIRGA42.mean,RHIRGA84.mean,T84.std
date,,,,,,,,,,,,,,,
2000-01-17,0.144030,4.448289,0.141368,4.351579,4.349342,4.422000,0.145686,0.082070,0.627895,0.084807,0.256133,74.066842,74.900921,74.509200,0.271648
2000-02-28,0.201369,7.251721,0.186227,7.166230,7.121724,7.219590,0.181269,0.840141,3.828525,0.836184,1.926296,99.753984,99.619180,99.240574,0.271090
2000-03-24,0.252571,3.621523,0.240205,3.540331,3.539536,3.592039,0.248413,0.194925,0.892318,0.193543,1.404963,57.530132,56.786623,56.476316,2.334741
2000-03-30,0.520673,6.515472,0.517986,6.512152,6.519430,6.512642,0.519211,0.490764,2.263544,0.460845,1.737499,68.467025,67.020000,67.095220,2.695277
2000-04-04,0.199853,6.269451,0.204433,6.218667,6.223394,6.240667,0.203877,0.879900,3.078545,0.873474,1.597401,89.108364,86.910732,86.864424,1.614122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-08-16,0.255920,15.224194,0.279175,15.026299,14.995519,15.189351,0.273949,0.173676,0.282138,0.169976,0.898936,78.929675,77.138323,77.203052,1.308728
2011-08-19,0.969445,11.328859,0.963615,10.641908,10.550927,11.183046,0.972549,0.528703,0.423590,0.382359,0.463660,60.114079,63.270067,62.037815,3.055122
2011-08-21,0.565227,13.218322,0.604045,13.013600,12.995638,13.185101,0.578579,0.294129,0.330323,0.285194,0.575115,77.896242,77.170134,76.968926,1.745526


In [17]:
estimator = svm.SVC(kernel="linear")
selectorM = RFECV(estimator, step=1, cv=5)
selectorM = selectorM.fit(X, y4)
df4_bestFeatures = X.loc[: , selectorM.support_]
df4_bestFeatures

,H2O42.mean,H2O42.std,H2O672.mean,NOx168.std,NOx42.std,NOx672.mean,NOx672.std,O3168.mean,O342.mean,O3504.std,O384.std,RHIRGA168.mean,RHIRGA168.std,RHIRGA336.mean,RHIRGA504.mean,RHIRGA84.std,SO2168.mean,SO2168.std,T42.std,T672.std
date,,,,,,,,,,,,,,,,,,,,
2000-01-17,4.448289,0.141368,4.349342,0.082070,0.084344,0.623947,0.084807,39.497763,39.225789,0.622920,0.586287,74.345921,3.036624,74.066842,74.543421,3.153985,0.114474,0.106350,0.279456,0.250952
2000-02-28,7.251721,0.186227,7.121724,0.840141,0.821353,3.830902,0.836184,31.483770,31.185410,1.554540,1.579364,100.283197,0.945387,99.753984,100.227623,0.816601,0.301803,0.121227,0.258797,0.313460
2000-03-24,3.621523,0.240205,3.539536,0.194925,0.216128,0.898411,0.193543,43.878487,43.602185,1.864525,1.982369,57.051184,14.097889,57.530132,58.060066,14.347865,0.116250,0.108144,2.401763,2.037111
2000-03-30,6.515472,0.517986,6.519430,0.490764,0.516708,2.268101,0.460845,45.279308,45.071258,2.419174,2.433812,67.839057,8.753772,68.467025,69.449051,8.855484,0.316289,0.141688,2.738091,2.509758
2000-04-04,6.269451,0.204433,6.223394,0.879900,0.883339,3.060667,0.873474,40.582303,40.071524,1.842615,1.952212,88.100848,8.233274,89.108364,90.234727,8.428027,0.692000,0.222253,1.589103,1.438071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-08-16,15.224194,0.279175,14.995519,0.173676,0.164278,0.288050,0.169976,22.566918,21.442187,6.707645,6.462438,77.766299,5.722857,78.929675,79.537987,5.952540,0.163403,0.103985,1.317451,1.089563
2011-08-19,11.328859,0.963615,10.550927,0.528703,0.443383,0.366859,0.382359,28.490641,25.917692,6.501999,6.404498,60.188278,16.640822,60.114079,59.979145,16.718655,0.118561,0.097648,3.031136,2.868514
2011-08-21,13.218322,0.604045,12.995638,0.294129,0.264188,0.333247,0.285194,19.005909,18.084286,6.703358,6.562469,77.193490,6.026740,77.896242,78.177800,5.911362,0.077373,0.100453,1.731175,1.696894


In [18]:
X2_bf_train, X2_bf_test, y2_bf_train, y2_bf_test = train_test_split(df2_bestFeatures, y2, test_size=0.2, random_state=0)
X4_bf_train, X4_bf_test, y4_bf_train, y4_bf_test = train_test_split(df4_bestFeatures, y4, test_size=0.2, random_state=0)


In [19]:
kernel_types = ["linear", "poly", "rbf", "sigmoid"]
resultsBinaryBf = pd.DataFrame(columns = ['Kernel_type', 'Accuracy', 'Precision', 'Recall'])

for k in kernel_types:
    clf = svm.SVC(kernel=k) 
    clf.fit(X2_bf_train, y2_bf_train)
    y2_bf_pred = clf.predict(X2_bf_test)
    resultsBinaryBf.loc[len(resultsBinaryBf.index)] = [k, 
                                       metrics.accuracy_score(y2_bf_test, y2_bf_pred), 
                                       metrics.precision_score(y2_bf_test, y2_bf_pred),
                                       metrics.recall_score(y2_bf_test, y2_bf_pred)]
resultsBinaryBf

,Kernel_type,Accuracy,Precision,Recall
0,linear,0.838710,0.886364,0.795918
1,poly,0.806452,0.816327,0.816327
2,rbf,0.849462,0.857143,0.857143
3,sigmoid,0.247312,0.243902,0.204082


In [20]:
kernel_types = ["linear", "poly", "rbf", "sigmoid"]
resultsMultiClassBf = pd.DataFrame(columns = ['Kernel_type', 'Accuracy'])

for k in kernel_types:
    clf = svm.SVC(kernel=k) 
    clf.fit(X4_bf_train, y4_bf_train)
    y4_bf_pred = clf.predict(X4_bf_test)
    resultsMultiClassBf.loc[len(resultsMultiClassBf.index)] = [k, metrics.accuracy_score(y4_bf_test, y4_bf_pred)]
    
resultsMultiClassBf

,Kernel_type,Accuracy
0,linear,0.655914
1,poly,0.580645
2,rbf,0.559140
3,sigmoid,0.419355


In [27]:
param_grid = {'C':[0.1,1,2,5], 'kernel':["linear", "poly" "rbf", "sigmoid"],'degree':[1,2,3,4,5]}
grid = GridSearchCV(svm.SVC(), param_grid)
grid.fit(X4_bf_train, y4_bf_train)

print(grid.best_params_)

{'C': 2, 'degree': 1, 'kernel': 'linear'}


/opt/homebrew/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
100 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.10/site-packages/sklearn/svm/_base.py", line 251, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/opt/homebrew/lib/python3.10/site-packages/sklearn/svm/_base.py", line 333, in _dense_fit
    ) = libsvm.fit(
  File "sklearn/svm/_libsvm.py

In [28]:
clf = svm.SVC(C=2, kernel='linear') 
clf.fit(X4_bf_train, y4_bf_train)
y4_bf_pred_final = clf.predict(X4_bf_test)
metrics.accuracy_score(y4_bf_test, y4_bf_pred_final)

0.6559139784946236

In [31]:
clf = svm.SVC(C=2, kernel='linear') 
clf.fit(X2_bf_train, y2_bf_train)
y2_bf_pred_final = clf.predict(X2_bf_test)
metrics.accuracy_score(y2_bf_test, y2_bf_pred_final)

0.8494623655913979